In [1]:
import numpy as np
import pandas as pd
from layer import *
from network import *

In [2]:
# dataset and preprocessing imports, no ML models imported
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and prepare data

In [3]:
def load_dataset():
    data = load_iris()
    df = pd.DataFrame(data["data"], columns=data["feature_names"])
    df["target"] = [data["target_names"][target_idx] for target_idx in data["target"]]
    return df

In [4]:
df = load_dataset()
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
def prepare_data(df):
    df = df.copy()
    df = df.rename(columns={"sepal length (cm)": "s_length",
                            "sepal width (cm)": "s_width",
                            "petal length (cm)": "p_length",
                            "petal width (cm)": "p_width"
    })
    df = pd.get_dummies(df, columns=["target"], dtype=int)
    return df
    
df = prepare_data(df)
df.head()

,s_length,s_width,p_length,p_width,target_setosa,target_versicolor,target_virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


#### Check null values and data types

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   s_length           150 non-null    float64
 1   s_width            150 non-null    float64
 2   p_length           150 non-null    float64
 3   p_width            150 non-null    float64
 4   target_setosa      150 non-null    int64  
 5   target_versicolor  150 non-null    int64  
 6   target_virginica   150 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 8.3 KB


# Train-test split and scaling X between 0 and 1

In [7]:
def Xy_split(df):
    df = df.copy()
    y_cols = ["target_setosa", "target_versicolor", "target_virginica"]
    y = df[y_cols]
    X = df.drop(y_cols, axis=1)
    return X,y

In [8]:
X,y = Xy_split(df)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# assure that data is converted to np.array
X_train, X_test, y_train, y_test = [np.array(a) for a in [X_train, X_test, y_train, y_test]]

In [10]:
def scale_data(X_train, X_test):
    X_train, X_test = X_train.copy(), X_test.copy()
    scaler = MinMaxScaler()
    # fit the scaler only using X_train
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [11]:
X_train, X_test = scale_data(X_train, X_test)

In [12]:
def transpose_data(X_train, X_test, y_train):
    # necessary for this specific neural net
    # y_test doesn't need to be transposed
    X_train, X_test, y_train = X_train.copy(), X_test.copy(), y_train.copy()
    X_train, X_test, y_train = X_train.T, X_test.T, y_train.T
    return X_train, X_test, y_train

In [13]:
X_train, X_test, y_train = transpose_data(X_train, X_test, y_train)

In [14]:
def get_Ninputs_Noutputs(X_train, y_train):
    # each column is a sample (already transposed)
    n_inputs = X_train.shape[0]
    n_outputs = y_train.shape[0]
    return n_inputs, n_outputs

In [15]:
n_inputs, n_outputs = get_Ninputs_Noutputs(X_train, y_train)

# Train Model

In [16]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

In [18]:
hidden1 = HiddenLayer([n_inputs, 20], relu, relu_derivative)
out = LastLayer([20,n_outputs], sigmoid, sigmoid_derivative)

net = Network([hidden1, out])

In [19]:
net.fit(X_train, y_train, learning_rate=1, epochs=100, batch_size=10)

Epoch: 99	Error:0.137774056494906045

In [20]:
y_pred = net.forward_propagate(X_test)[-1]
y_pred = y_pred.T

In [21]:
def get_category(y_sample):
    cont_y = list(y_sample)
    max_value = max(y_sample)
    max_idx = cont_y.index(max_value)
    return max_idx

In [22]:
y_pred = [get_category(y_sample) for y_sample in y_pred]
y_test = [get_category(y_sample) for y_sample in y_test]

# Results

In [23]:
print("Real:     ",y_test)
print("Predicted:",y_pred)
print("Accuracy =",round(accuracy_score(y_test, y_pred)*100, 2), "%")

Real:      [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Predicted: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy = 100.0 %
